CONTROLE DE AVISOS

In [2]:
import warnings
#Essa linha serve para ignorar os avisos que aparecem no código
warnings.filterwarnings("ignore")

INSTALAÇÃO DAS BIBLIOTECAS NECESSÁRIAS

In [ ]:
%pip install crewai_tools

IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
import os

In [5]:
#Definindo a chave de API do OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#Definindo o modelo do ChatGPT que será usado
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

CRIAÇÃO DE AGENTES

In [6]:
#Definindo o agente de suporte
support_agent = Agent(
    #Definindo a função do agente
    role="Senior Support Representative",
    #Definindo o objetivo do agente
    goal="Be the most friendly and helpful support representative in your team",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working on providing"
        "support to {customer}, a super important costumer for your company."
        "You need to make sure that you provide the best support!"
        "Make sure to provide full complete answers, and make no assumptions."
    ),
    #Esse agente não podera delegar tarefas para outros
    allow_delegation=False,
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

In [7]:
#Definindo o agente de qualidade de suporte, esse agente pode delegar tarefas
support_quality_assurance_agent = Agent(
    #Definindo a função do agente
    role="Support Quality Assurance Specialist",
    #Definindo o objetivo do agente
    goal="Get recognition for providing the best support quality assurance in your team",
    #Aqui é definida a "identidade" do agente,
    #ou seja, o modo como ele irá agir
    backstory=(
        "You work at crewAI (https://crewai.com) and are now working with your team on"
        "a request from {customer} ensuring that the support representative"
        "is providing the best support possible.\n"
        "You need to make sure that the support representative is providing"
        "full complete answers, and make no assumptions."
    ),
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True
)

Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

DEFINIÇÃO DAS FERRAMENTAS

In [8]:
#Definindo uma ferramenta para fazer buscas no Google
search_tool = SerperDevTool()
#Definindo uma ferramenta para retirar as informações de um site
scrape_tool = ScrapeWebsiteTool()
#Definindo uma ferramenta para fazer buscas em um site
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

Different Ways to Give Agents Tools
- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific task

Note: Tasks Tools override the Agent Tools

CRIAÇÃO DAS TASKS

In [9]:
inquiry_resolution = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, o agente deve responder a uma pergunta de um cliente
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. Make sure "
        "to use everything you know to provide the best support possible."
        "You must strive to provide a complete and accurate response to the costumer's  inquiry."
    ),
    #Aqui é definido o output esperado, o que esperamos que o agente retorne
    expected_output=(
        "A detailed, informative response to the costumer's "
        "inquiry that addresses all aspects of their question.\n"
        "The response should include references to everything "
        "you used to find the answer, including external data or solutions."
        "Ensure the answer is complete, leaving no questions unanswered,"
        "and maitain a helpful and friendly tone throughout."
    ),
    #Aqui é definida as ferramentas que o agente pode usar para realizar a tarefa
    tools=[docs_scrape_tool],
    #Aqui é definido o agente que irá realizar a tarefa
    agent=support_agent,
)

In [10]:
quality_assurance_review = Task(
    #Aqui é definida a descrição da tarefa, o que esperamos que o agente faça, a tarefa planejada
    #Nesse caso, o agente deve revisar a resposta feita pelo Representante de Suporte Sênior
    description=(
        "Review the response drafted by the Senior Support Representative"
        "Ensure that the answer is comprehensive, accurate, and adheres to the"
        "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry have been addressed "
        "thoroughly, with a helful and friendly tone.\n"
        "Check for references and sources used to find the information,"
        "ensuring the response is well-supported and leaves no questions unanswered."
    ),
    #Aqui é definido o resultado esperado da tarefa, o que esperamos que o agente entregue
    expected_output=(
        "A final, detailed, and informative response ready to be sent to the costumer.\n"
        "This response should fully address the costumer's inquiry,"
        "incoporating all relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company, "
        "but maintain a professional and friendly tone throughout."
    ),
    #Aqui é a atribuição da tarefa para o respectivo agente 
    agent=support_quality_assurance_agent,
)

CRIAÇÃO DO CREW (GRUPO DOS AGENTES QUE TRABALHAM JUNTOS)

In [11]:
crew = Crew(
    #Adicionando os agentes ao Crew
    agents=[support_agent, support_quality_assurance_agent],
    #Adicionando as tarefas ao Crew
    tasks=[inquiry_resolution, quality_assurance_review],
    #Parâmetro que define se as mensagens sobre as 
    #ações realizadas pelos agentes serão exibidas
    verbose=True,
    #Ativa todas as memórias disponíveis no CrewAI
    memory=True,
)

CHAMANDO A FUNÇÃO CREW PARA QUE O SISTEMA SEJA REALIZADO

In [12]:
#Definindo os valores de entrada para o Crew, 
#os valores que preencheram os campos/variáveis dos "prompts"
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew and kicking it off,"
    " specifically how can I add memory to my crew? Can you provide guidance?"
}
#Iniciando o Crew
result = crew.kickoff(inputs=inputs)

 
[2025-03-18 21:20:25][🚀 CREW 'CREW' STARTED, 3BC2E973-72DB-4289-BF57-2B28ABDE5F31]: 2025-03-18 21:20:25.990491
 
[2025-03-18 21:20:26][📋 TASK STARTED: DEEPLEARNINGAI JUST REACHED OUT WITH A SUPER IMPORTANT ASK:
I NEED HELP WITH SETTING UP A CREW AND KICKING IT OFF, SPECIFICALLY HOW CAN I ADD MEMORY TO MY CREW? CAN YOU PROVIDE GUIDANCE?

ANDREW NG FROM DEEPLEARNINGAI IS THE ONE THAT REACHED OUT. MAKE SURE TO USE EVERYTHING YOU KNOW TO PROVIDE THE BEST SUPPORT POSSIBLE.YOU MUST STRIVE TO PROVIDE A COMPLETE AND ACCURATE RESPONSE TO THE COSTUMER'S  INQUIRY.]: 2025-03-18 21:20:26.015139
 
[2025-03-18 21:20:27][🤖 AGENT 'SENIOR SUPPORT REPRESENTATIVE' STARTED TASK]: 2025-03-18 21:20:27.947759
# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sur